# Matrix multiplication

Matrix multiplication is one of the most well-known linear algebra algorithms, and frequently used to demonstrate the high-performance computing capabilities of GPUs. As such, an example using matrix multiplication could not be left out. A naive CUDA kernel for a square matrix multiplication is:

In [ ]:
# %load matmul_naive.cu
#define WIDTH 4096

__global__ void matmul_kernel(float *C, float *A, float *B) {
    int x = blockIdx.x * block_size_x + threadIdx.x;
    int y = blockIdx.y * block_size_y + threadIdx.y;
    float sum = 0.0;

    for (int k=0; k<WIDTH; k++) {
        sum += A[y*WIDTH+k] * B[k*WIDTH+x];
    }

    C[y*WIDTH+x] = sum;
}


This kernel simply creates a single thread per output element. Each thread computes the index of the element it is responsible for, and iterates over the corresponding row in A, and corresponding column in B.

In [32]:
#!/usr/bin/env python
import numpy
import kernel_tuner
from collections import OrderedDict

problem_size = (4096, 4096)
size = numpy.prod(problem_size)

A = numpy.random.randn(*problem_size).astype(numpy.float32)
B = numpy.random.randn(*problem_size).astype(numpy.float32)
C = numpy.zeros_like(A)

args = [C, A, B]
tune_params = OrderedDict()
tune_params["block_size_x"] = [16*2**i for i in range(3)]
tune_params["block_size_y"] = [2**i for i in range(6)]

answer = [numpy.dot(A,B), None, None]

results = kernel_tuner.tune_kernel("matmul_kernel", "matmul_naive.cu",
                                   problem_size, args, tune_params, answer=answer, atol=1e-3)  

Using: GeForce GTX TITAN X
block_size_x=16, block_size_y=1, time=1649.89187012
block_size_x=16, block_size_y=2, time=832.841296387
block_size_x=16, block_size_y=4, time=793.8703125
block_size_x=16, block_size_y=8, time=728.78170166
block_size_x=16, block_size_y=16, time=574.705957031
block_size_x=16, block_size_y=32, time=494.258599854
block_size_x=32, block_size_y=1, time=977.559960938
block_size_x=32, block_size_y=2, time=867.61730957
block_size_x=32, block_size_y=4, time=830.580737305
block_size_x=32, block_size_y=8, time=623.53215332
block_size_x=32, block_size_y=16, time=548.464074707
block_size_x=32, block_size_y=32, time=514.333398438
block_size_x=64, block_size_y=1, time=963.868127441
block_size_x=64, block_size_y=2, time=952.567932129
block_size_x=64, block_size_y=4, time=789.973242187
block_size_x=64, block_size_y=8, time=678.554589844
block_size_x=64, block_size_y=16, time=602.40604248
best performing configuration: block_size_x=16, block_size_y=32, time=494.258599854


There aren't many parameters to tune yet, and more importantly, tuning will not be very effective because this kernel will be limited by bandwidth rather than compute. There is however, a lot of opportunity for data reuse, which is realized by making the threads in a thread block collaborate.

The utilisation of the GPU is still very low:

![](Matmul-naive-utilisation.png)

# Increase data reuse

This can be solved by using a technique called loop-blocking or loop-tiling. We define two square data structures in shared memory, which will be used for storing square parts of matrix A and B. The threads in a thread block will collaboratively fill these two variables, and then proceed to perform all the computations that need this data, before moving to the next blocked iteration.

In [ ]:
# %load matmul_data_reuse.cu
__global__ void matmul_kernel(float *C, float *A, float *B) {

    __shared__ float sA[block_size][block_size];
    __shared__ float sB[block_size][block_size];

    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int x = blockIdx.x * block_size + tx;
    int y = blockIdx.y * block_size + ty;

    float sum = 0.0;
    int k,kb;

    for (k=0; k<WIDTH; k+=block_size) {
        __synchthreads();
        sA[ty][tx] = A[y*WIDTH+k+tx];
        sB[ty][tx] = B[(k+ty)*WIDTH+x];
        __synchthreads();

        for (kb=0; kb<block_size; kb++) {
            sum += sA[ty][kb] * sB[kb][tx];
        }

    }

    C[y*WIDTH+x] = sum;
}

